In [1]:
!pip install pytorchvideo
!pip install torchsummary

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 KB 978.8 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 KB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.2/28.2 MB 28.3 MB/s eta 0:00:00
  Created wheel for pytorchvideo: filename=pytorchvideo-0.1.5-py3-none-any.whl size=188714 sha256=f086138c6457dba63b0dd259d8805b9edef9e3829fa8bca2047c60448cf5937c
  Stored in directory: /root/.cache/pip/wheels/e8/51/05/053b29bac2400cbbae2fb7cfc41afd280d627bca7c9363ca80
  Created wheel for fvcore: filename=fvcore-0.1.5.post20220512-py3-none-any.whl size=61288 sha256=1a91fd7fa0de9f70e3510b6ed7b4ae43b88ae16c08ec64097f226eaacfeecc68
  Stored in directory: /root/.cache/pip/wheels/68/20/f9/a11a0dd63f4c13678b2a5ec488e48078756505c7777b75b29e
Successfully built pytorchvideo fvcore


In [2]:
import torch
import numpy as np
from torch.utils.data import (
    Dataset,
    DataLoader,
) 
import pickle


import os

import math
import torch.nn as nn
import torch.nn.functional as F

from torchvision.transforms import Compose, Lambda, Grayscale,Normalize, CenterCrop,Resize
#from torchvision.transforms._transforms_video import CenterCropVideo
from pytorchvideo.data.encoded_video import EncodedVideo
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    UniformTemporalSubsample,
)

from tqdm import tqdm

from collections import OrderedDict

import torch.optim as optim

from torch.autograd import Variable


import torchvision.models as models

import sys
from torchsummary import summary






class SignDataset(Dataset):
    def __init__(self,x,y):
        self.x = x
        self.y = y


    def __len__(self):

        return len(self.y)

        # length = 0
        # with open(self.file, 'rb') as f:
        #     data = pickle.load(f)

        # for x in data:
        #     length += len(data[x])
        # return length


    def __getitem__(self, index):
        return self.x[index], self.y[index]
        

In [3]:
IMAGE_HEIGHT = 720
IMAGE_WIDTH = 800
IMAGE_CHANNEL = 1
NUM_FRAMES = 25
NUM_CLASSES = 60
neg = 0





def transform_data(x):
    
    transform =  ApplyTransformToKey(
        key="video",
        transform=Compose(
            [
                Lambda(lambda x: x/255.0),
                Normalize(([0.2948, 0.4811, 0.6757]), ([0.1593, 0.1641, 0.2440])),
                Grayscale(num_output_channels=1),
                CenterCrop([720,900]),
                Resize([512,512]),
                Lambda(lambda x: x.permute(1,2,3,0)),#(channel, frames(depth), height, width)

            ]

        ),
    )
    
    return transform(x)

def lp_video(video,start_time, end_time):
    video_data = video.get_clip(start_sec=float(start_time)/1000.0, end_sec=float(end_time)/1000.0)
            #print(video_data["video"].shape)
 
    if video_data["video"] is None:
        return None        # or pass
    else:
    
        #video_data["video"] = Grayscale(num_output_channels=1)((video_data["video"]).permute(1,0,2,3))
    #             video_data["video"] = video_data["video"]/255
                #print(video_data["video"].shape)
        video_data["video"] = video_data["video"].permute(1,0,2,3)
#         std, mean = torch.std_mean(video_data["video"],dim=[0,2,3])
#         std = std/255.0
#         #print(std)
#         mean = mean/255.0
        #print(mean)
        video_data = transform_data( video_data)

        return video_data["video"]

def load_dataloaders(batch_size,start,end):

    time_start = []
    time_end = []
    train_inputs =[] #x
    train_classes = [] #y
    global neg

    with open('../input/signdataset/sign/MSSL_Train_Set/TRAIN/MSSL_TRAIN_SET_GT.pkl', 'rb') as f:
        data = pickle.load(f)


# keys are files so iterate only limited files due to memory limitations.
    for key in list(data.keys())[start:end]:
        filename = key
        print("file",filename)
        video = EncodedVideo.from_path("../input/signdataset/sign/MSSL_Train_Set/TRAIN/MSSL_TRAIN_SET_VIDEOS_ELAN/"+filename+".mp4")
    # file functions

        for x in data[key]:
            img_cls = x[0]
            time_start.append(x[1])
            time_end.append(x[2])
            
            
            # start_time = x[1]
            # end_time = x[2]
            vid = lp_video(video,x[1], x[2])

            for m in torch.unbind(vid, dim=3):
                train_classes.append(img_cls)
                train_inputs.append(m)


            #some negative classes too
        for i in range(len(time_start)-2):
            if (time_start[i+1]- time_end[i])>1000.0:
                start_t = time_end[i]+800.0
                end_t = time_end[i]+920.0
                vid = lp_video(video,start_t, end_t)
                if vid is None:
                    break
                else:
                    for m in torch.unbind(vid, dim=3):
                        if neg <= 1300:
                            train_classes.append(60)
                            train_inputs.append(m)
                            neg += 1
        print(neg)
            
        
            

    signds = SignDataset(train_inputs, train_classes)
    trainlen = int(len(signds)*0.8)
    torch.manual_seed(0)
    
    train_set, val_test_set = torch.utils.data.random_split(signds, [trainlen, len(signds)-trainlen])
    trainloader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)

    valloader = DataLoader(val_test_set, batch_size=batch_size, shuffle=True, num_workers=2)
  
    return trainloader, valloader
        
    

In [4]:
def load_testloader(batch_size,start,end):

    time_start = []
    time_end = []
    train_inputs =[] #x
    train_classes = [] #y
    

    with open('../input/signdataset/sign/MSSL_Train_Set/TRAIN/MSSL_TRAIN_SET_GT.pkl', 'rb') as f:
        data = pickle.load(f)


# keys are files so iterate only limited files due to memory limitations.
    for key in list(data.keys())[start:end]:
        filename = key
        print("file",filename)
        video = EncodedVideo.from_path("../input/signdataset/sign/MSSL_Train_Set/TRAIN/MSSL_TRAIN_SET_VIDEOS_ELAN/"+filename+".mp4")
    # file functions

        for x in data[key]:
            img_cls = x[0]
            time_start.append(x[1])
            time_end.append(x[2])
            
            
            # start_time = x[1]
            # end_time = x[2]
            vid = lp_video(video,x[1], x[2])

            for m in torch.unbind(vid, dim=3):
                train_classes.append(img_cls)
                train_inputs.append(m)


            #some negative classes too
        for i in range(0,len(time_start)//2):
            if (time_start[i+1]- time_end[i])>1000.0:
                start_t = time_end[i]+700.0
                end_t = time_end[i]+780.0
                vid = lp_video(video,start_t, end_t)
                if vid is None:
                    break
                else:
                    for m in torch.unbind(vid, dim=3):
                        train_classes.append(60)
                        train_inputs.append(m)

            
        
            

    signds = SignDataset(train_inputs, train_classes)
    
    

    testloader = DataLoader(signds, batch_size=batch_size, shuffle=True, num_workers=2)

    return testloader

In [5]:
def load_vid(batch_size,start,end):


    train_inputs =[] #x

    video = EncodedVideo.from_path("../input/signdataset/sign/MSSL_Val_Set/VALIDATION/MSSL_VAL_SET_VIDEOS/p01_n131.mp4")
    # file functions

    vid = lp_video(video,start, end)

    for m in torch.unbind(vid, dim=3):
        train_inputs.append(m)

    

    testloader = DataLoader(train_inputs, batch_size=batch_size, shuffle=True, num_workers=2)

    return testloader

In [6]:
"""
Creates a EfficientNetV2 Model as defined in:
Mingxing Tan, Quoc V. Le. (2021). 
EfficientNetV2: Smaller Models and Faster Training
arXiv preprint arXiv:2104.00298.
import from https://github.com/d-li14/mobilenetv2.pytorch
"""



__all__ = ['effnetv2_s', 'effnetv2_m', 'effnetv2_l', 'effnetv2_xl']


def _make_divisible(v, divisor, min_value=None):
    """
    This function is taken from the original tf repo.
    It ensures that all layers have a channel number that is divisible by 8
    It can be seen here:
    https://github.com/tensorflow/models/blob/master/research/slim/nets/mobilenet/mobilenet.py
    :param v:
    :param divisor:
    :param min_value:
    :return:
    """
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v


# SiLU (Swish) activation function
if hasattr(nn, 'SiLU'):
    SiLU = nn.SiLU
else:
    # For compatibility with old PyTorch versions
    class SiLU(nn.Module):
        def forward(self, x):
            return x * torch.sigmoid(x)

 
class SELayer(nn.Module):
    def __init__(self, inp, oup, reduction=4):
        super(SELayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
                nn.Linear(oup, _make_divisible(inp // reduction, 8)),
                SiLU(),
                nn.Linear(_make_divisible(inp // reduction, 8), oup),
                nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y


def conv_3x3_bn(inp, oup, stride):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
        nn.BatchNorm2d(oup),
        SiLU()
    )


def conv_1x1_bn(inp, oup):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
        nn.BatchNorm2d(oup),
        SiLU()
    )


class MBConv(nn.Module):
    def __init__(self, inp, oup, stride, expand_ratio, use_se):
        super(MBConv, self).__init__()
        assert stride in [1, 2]

        hidden_dim = round(inp * expand_ratio)
        self.identity = stride == 1 and inp == oup
        if use_se:
            self.conv = nn.Sequential(
                # pw
                nn.Conv2d(inp, hidden_dim, 1, 1, 0, bias=False),
                nn.BatchNorm2d(hidden_dim),
                SiLU(),
                # dw
                nn.Conv2d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False),
                nn.BatchNorm2d(hidden_dim),
                SiLU(),
                SELayer(inp, hidden_dim),
                # pw-linear
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
            )
        else:
            self.conv = nn.Sequential(
                # fused
                nn.Conv2d(inp, hidden_dim, 3, stride, 1, bias=False),
                nn.BatchNorm2d(hidden_dim),
                SiLU(),
                # pw-linear
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
            )


    def forward(self, x):
        if self.identity:
            return x + self.conv(x)
        else:
            return self.conv(x)


class EffNetV2(nn.Module):
    def __init__(self, cfgs, num_classes=61, width_mult=1.):
        super(EffNetV2, self).__init__()
        self.cfgs = cfgs

        # building first layer
        input_channel = _make_divisible(24 * width_mult, 8)
        layers = [conv_3x3_bn(1, input_channel, 2)]
        # building inverted residual blocks
        block = MBConv
        for t, c, n, s, use_se in self.cfgs:
            output_channel = _make_divisible(c * width_mult, 8)
            for i in range(n):
                layers.append(block(input_channel, output_channel, s if i == 0 else 1, t, use_se))
                input_channel = output_channel
        self.features = nn.Sequential(*layers)
        # building last several layers
        output_channel = _make_divisible(1792 * width_mult, 8) if width_mult > 1.0 else 1792
        self.conv = conv_1x1_bn(input_channel, output_channel)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Linear(output_channel, num_classes)

        self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = self.conv(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.001)
                m.bias.data.zero_()


def effnetv2_s(**kwargs):
    """
    Constructs a EfficientNetV2-S model
    """
    cfgs = [
        # t, c, n, s, SE
        [1,  24,  2, 1, 0],
        [4,  48,  4, 2, 0],
        [4,  64,  4, 2, 0],
        [4, 128,  6, 2, 1],
        [6, 160,  9, 1, 1],
        [6, 256, 15, 2, 1],
    ]
    return EffNetV2(cfgs, **kwargs)


def effnetv2_m(**kwargs):
    """
    Constructs a EfficientNetV2-M model
    """
    cfgs = [
        # t, c, n, s, SE
        [1,  24,  3, 1, 0],
        [4,  48,  5, 2, 0],
        [4,  80,  5, 2, 0],
        [4, 160,  7, 2, 1],
        [6, 176, 14, 1, 1],
        [6, 304, 18, 2, 1],
        [6, 512,  5, 1, 1],
    ]
    return EffNetV2(cfgs, **kwargs)


def effnetv2_l(**kwargs):
    """
    Constructs a EfficientNetV2-L model
    """
    cfgs = [
        # t, c, n, s, SE
        [1,  32,  4, 1, 0],
        [4,  64,  7, 2, 0],
        [4,  96,  7, 2, 0],
        [4, 192, 10, 2, 1],
        [6, 224, 19, 1, 1],
        [6, 384, 25, 2, 1],
        [6, 640,  7, 1, 1],
    ]
    return EffNetV2(cfgs, **kwargs)


def effnetv2_xl(**kwargs):
    """
    Constructs a EfficientNetV2-XL model
    """
    cfgs = [
        # t, c, n, s, SE
        [1,  32,  4, 1, 0],
        [4,  64,  8, 2, 0],
        [4,  96,  8, 2, 0],
        [4, 192, 16, 2, 1],
        [6, 256, 24, 1, 1],
        [6, 512, 32, 2, 1],
        [6, 640,  8, 1, 1],
    ]
    return EffNetV2(cfgs, **kwargs)

In [7]:

def evluate(model, device, loader, mode="Validate"):
    
    
    model.eval()

    correct = 0
    num_samples = 0
    
        
    for batch_idx, data in enumerate(loader):
        with torch.no_grad():    
            data = data.to(device)
            

            output = model(data.half())  
            
            _,pred = output.max(1)# get the index of the max log-probability
            print(pred)



In [8]:

def test(model, device, loader, mode="Validate"):
    
    
    model.eval()

    correct = 0
    num_samples = 0
    
        
    for batch_idx, (data, target) in enumerate(loader):
        with torch.no_grad():    
            data = data.to(device)
            target = (target).to(device)

            output = model(data.half())  
            
            _,pred = output.max(1)# get the index of the max log-probability
            num_samples += target.size(0)
            correct += (pred==target).sum().item()
            #print("correct", correct)

    acc = 100.0 * correct / num_samples
    
    print('{} Accuracy: {}/{} ({:.0f}%)'.format(
                mode,correct, num_samples,
                100. * correct / num_samples, acc))

In [9]:

    


def train(model, device, train_loader,validloader,scheduler, optimizer,criterion,epochs,lr):
    print("Train start")
    breakout = False
    model.half()
    model.cuda()

    
    model.train()

    optimizer.param_groups[0]['lr'] = lr
    
    for epoch in range(epochs):
        correct = 0
        num_samples = 0
        train_loss = 0
        
        for batch_idx, (data, target) in enumerate(train_loader):
            
            data = data.to(device)
            target = target.to(device)

            optimizer.zero_grad()
            output = model(data.half())
            
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            
            train_loss += loss.item()
            
            
            _,pred = output.max(1)# get the index of the max log-probability
            num_samples += target.size(0)
            correct += (pred==target).sum().item()
            
        if optimizer.param_groups[0]['lr'] != 1e-3:    
            scheduler.step()
        #print("lr:",optimizer.param_groups[0]['lr'])
        
        
        train_loss /= num_samples
        
        if (100 * correct / num_samples) >= 95:
            
            print('Epoch: {} , Training Accuracy: {}/{} ({:.0f}%) Training Loss: {:.6f}'.format(
                epoch+1, correct, num_samples,100. * correct / num_samples, train_loss))
            
            test(model,device,validloader, mode = "Validating")
#             test(model, device, testloader, mode = "Test before Training on validation set")
           # model, optimizer = trainonval(model, device, validloader, optimizer, criterion, epochs)
#             test(model, device, testloader, mode = "Test after training on validation set")
            
            
            
            torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()}, "./model_optimizer34.pt")
            
            breakout = True
            print("breakout")
            return model, optimizer
            
            
            
       
        

        print('Epoch: {} , Training Accuracy: {}/{} ({:.0f}%) Training Loss: {:.6f}'.format(
                epoch+1, correct, num_samples,
                100. * correct / num_samples, train_loss))
        if (((epoch+1)%5) == 0) :
            test(model,device,validloader)
            model.train()
            
    if breakout:
            print("breakout")
            return
    
    
    
    #model, optimizer = trainonval(model, device, validloader, optimizer, criterion, epochs)
    test(model, device, validloader, mode= "test set ")
    
    
        
    torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()}, "./model_optimizer34.pt")
    return model, optimizer
    
    
        


In [10]:
n_classes = 61


model = nn.Sequential(OrderedDict([
    ('frontend',effnetv2_m())
]))





# specify loss function (categorical cross-entropy)

# specify optimizer and learning rate
optimizer = optim.SGD(
    [
        
        {"params": model.frontend.parameters(), "lr": 1e-2},
  ],
  momentum = 0.9
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


samples_per_cls = [166.0, 370.0, 63.0, 232.0, 905.0, 102.0, 537.0, 993.0, 133.0, 154.0, 286.0, 435.0, 452.0, 779.0, 354.0, 136.0, 232.0, 2154.0, 374.0, 787.0, 337.0, 268.0, 954.0, 167.0, 543.0, 324.0, 669.0, 333.0, 211.0, 433.0, 140.0, 665.0, 107.0, 305.0, 162.0, 61.0, 1229.0, 958.0, 897.0, 543.0, 625.0, 334.0, 776.0, 111.0, 412.0, 583.0, 255.0, 103.0, 108.0, 201.0, 384.0, 632.0, 13.0, 322.0, 664.0, 435.0, 194.0, 322.0, 113.0, 462.0,2000.0]
no_of_classes=61
beta=0.9999
effective_num = 1.0 - np.power(beta, samples_per_cls)
weights = (1.0 - beta) / np.array(effective_num)
weights = weights / np.sum(weights) * no_of_classes
    

weights = torch.FloatTensor(weights).cuda()

criterion = nn.CrossEntropyLoss(weight = weights.half())

state = torch.load("../input/fmosel/model_optimizer34.pt")
model.load_state_dict(state['model_state_dict'])
model.half()
model.cuda()
optimizer.load_state_dict(state['optimizer_state_dict'])

In [11]:
#print('Number of model parameters: {}'.format(sum([p.data.nelement() for p in model.parameters()])))

In [12]:
#summary(model, (1, 512, 640), device='cpu')

In [13]:
trainloader, valloader = load_dataloaders(batch_size=16,start=0, end=6)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) 
model, optimizer = train(model,device,trainloader,valloader,scheduler,optimizer,criterion,30,1e-2)

file p01_n002
35
file p06_n019
122
file p06_n007
248
file p01_n053
422
file p04_n074
502
file p05_n077
746
Train start
Epoch: 1 , Training Accuracy: 719/1840 (39%) Training Loss: 0.134406
Epoch: 2 , Training Accuracy: 1308/1840 (71%) Training Loss: 0.047060
Epoch: 3 , Training Accuracy: 1516/1840 (82%) Training Loss: 0.026578
Epoch: 4 , Training Accuracy: 1619/1840 (88%) Training Loss: 0.015843
Epoch: 5 , Training Accuracy: 1665/1840 (90%) Training Loss: 0.014491
Validate Accuracy: 403/461 (87%)
Epoch: 6 , Training Accuracy: 1733/1840 (94%) Training Loss: 0.007759
Epoch: 7 , Training Accuracy: 1760/1840 (96%) Training Loss: 0.006111
Validating Accuracy: 421/461 (91%)
breakout


In [14]:
trainloader, valloader = load_dataloaders(batch_size=16,start=6, end=12)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) 
model, optimizer = train(model,device,trainloader,valloader,scheduler,optimizer,criterion,30,1e-2)

file p05_n027
786
file p05_n022
880
file p01_n110
962
file p06_n036
1047
file p01_n069
1141
file p06_n109
1223
Train start
Epoch: 1 , Training Accuracy: 619/1677 (37%) Training Loss: 0.143640
Epoch: 2 , Training Accuracy: 1155/1677 (69%) Training Loss: 0.040137
Epoch: 3 , Training Accuracy: 1412/1677 (84%) Training Loss: 0.017420
Epoch: 4 , Training Accuracy: 1482/1677 (88%) Training Loss: 0.012525
Epoch: 5 , Training Accuracy: 1569/1677 (94%) Training Loss: 0.007255
Validate Accuracy: 365/420 (87%)
Epoch: 6 , Training Accuracy: 1612/1677 (96%) Training Loss: 0.005220
Validating Accuracy: 380/420 (90%)
breakout


In [15]:
trainloader, valloader = load_dataloaders(batch_size=16,start=12, end=18)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) 
model, optimizer = train(model,device,trainloader,valloader,scheduler,optimizer,criterion,30,1e-2)

file p04_n089
1280
file p01_n044
1301
file p05_n037
1301
file p04_n038
1301
file p06_n055
1301
file p01_n099
1301
Train start
Epoch: 1 , Training Accuracy: 726/1331 (55%) Training Loss: 0.115861
Epoch: 2 , Training Accuracy: 1050/1331 (79%) Training Loss: 0.035707
Epoch: 3 , Training Accuracy: 1177/1331 (88%) Training Loss: 0.022559
Epoch: 4 , Training Accuracy: 1246/1331 (94%) Training Loss: 0.008467
Epoch: 5 , Training Accuracy: 1299/1331 (98%) Training Loss: 0.004915
Validating Accuracy: 306/333 (92%)
breakout


In [16]:
trainloader, valloader = load_dataloaders(batch_size=16,start=18, end=24)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) 
model, optimizer = train(model,device,trainloader,valloader,scheduler,optimizer,criterion,30,1e-2)

file p02_n066
1301
file p04_n035
1301
file p01_n057
1301
file p01_n105
1301
file p04_n058
1301
file p05_n043
1301
Train start
Epoch: 1 , Training Accuracy: 591/1017 (58%) Training Loss: 0.112199
Epoch: 2 , Training Accuracy: 933/1017 (92%) Training Loss: 0.015662
Epoch: 3 , Training Accuracy: 983/1017 (97%) Training Loss: 0.007398
Validating Accuracy: 251/255 (98%)
breakout


In [17]:
trainloader, valloader = load_dataloaders(batch_size=16,start=24, end=30)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) 
model, optimizer = train(model,device,trainloader,valloader,scheduler,optimizer,criterion,30,1e-3)

file p06_n020
1301
file p02_n001
1301
file p04_n032
1301
file p05_n084
1301
file p01_n102
1301
file p06_n096
1301
Train start
Epoch: 1 , Training Accuracy: 330/1236 (27%) Training Loss: 0.232928
Epoch: 2 , Training Accuracy: 695/1236 (56%) Training Loss: 0.096336
Epoch: 3 , Training Accuracy: 872/1236 (71%) Training Loss: 0.064310
Epoch: 4 , Training Accuracy: 949/1236 (77%) Training Loss: 0.048376
Epoch: 5 , Training Accuracy: 1041/1236 (84%) Training Loss: 0.036695
Validate Accuracy: 267/309 (86%)
Epoch: 6 , Training Accuracy: 1064/1236 (86%) Training Loss: 0.029889
Epoch: 7 , Training Accuracy: 1103/1236 (89%) Training Loss: 0.024797
Epoch: 8 , Training Accuracy: 1118/1236 (90%) Training Loss: 0.021729
Epoch: 9 , Training Accuracy: 1145/1236 (93%) Training Loss: 0.018516
Epoch: 10 , Training Accuracy: 1169/1236 (95%) Training Loss: 0.015621
Validate Accuracy: 285/309 (92%)
Epoch: 11 , Training Accuracy: 1163/1236 (94%) Training Loss: 0.013800
Epoch: 12 , Training Accuracy: 1188/1236

In [18]:
trainloader, valloader = load_dataloaders(batch_size=16,start=30, end=36)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1) 
model, optimizer = train(model,device,trainloader,valloader,scheduler,optimizer,criterion,30,1e-3)

file p02_n104
1301
file p01_n090
1301
file p01_n040
1301
file p04_n073
1301
file p01_n067
1301
file p05_n004
1301
Train start
Epoch: 1 , Training Accuracy: 499/1156 (43%) Training Loss: 0.152251
Epoch: 2 , Training Accuracy: 864/1156 (75%) Training Loss: 0.056329
Epoch: 3 , Training Accuracy: 996/1156 (86%) Training Loss: 0.033952
Epoch: 4 , Training Accuracy: 1046/1156 (90%) Training Loss: 0.024560
Epoch: 5 , Training Accuracy: 1068/1156 (92%) Training Loss: 0.018389
Validate Accuracy: 268/290 (92%)
Epoch: 6 , Training Accuracy: 1099/1156 (95%) Training Loss: 0.013454
Validating Accuracy: 269/290 (93%)
breakout


In [19]:
trainloader, valloader = load_dataloaders(batch_size=16,start=36, end=42)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) 
model, optimizer = train(model,device,trainloader,valloader,scheduler,optimizer,criterion,30,1e-3)

file p06_n061
1301
file p05_n092
1301
file p06_n015
1301
file p04_n106
1301
file p05_n095
1301
file p02_n072
1301
Train start
Epoch: 1 , Training Accuracy: 809/1322 (61%) Training Loss: 0.097048
Epoch: 2 , Training Accuracy: 1151/1322 (87%) Training Loss: 0.028669
Epoch: 3 , Training Accuracy: 1236/1322 (93%) Training Loss: 0.016512
Epoch: 4 , Training Accuracy: 1256/1322 (95%) Training Loss: 0.012194
Validating Accuracy: 315/331 (95%)
breakout


In [20]:
trainloader, valloader = load_dataloaders(batch_size=16,start=42, end=48)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1) 
model, optimizer = train(model,device,trainloader,valloader,scheduler,optimizer,criterion,30,1e-3)

file p05_n031
1301
file p01_n060
1301
file p05_n048
1301
file p05_n029
1301
file p01_n000
1301
file p01_n050
1301
Train start
Epoch: 1 , Training Accuracy: 609/1117 (55%) Training Loss: 0.129732
Epoch: 2 , Training Accuracy: 961/1117 (86%) Training Loss: 0.026497
Epoch: 3 , Training Accuracy: 1028/1117 (92%) Training Loss: 0.015863
Epoch: 4 , Training Accuracy: 1074/1117 (96%) Training Loss: 0.010852
Validating Accuracy: 265/280 (95%)
breakout


In [21]:
trainloader, valloader = load_dataloaders(batch_size=16,start=48, end=54)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1) 
model, optimizer = train(model,device,trainloader,valloader,scheduler,optimizer,criterion,30,1e-3)

file p05_n025
1301
file p04_n011
1301
file p01_n023
1301
file p01_n042
1301
file p01_n111
1301
file p05_n083
1301
Train start
Epoch: 1 , Training Accuracy: 721/1096 (66%) Training Loss: 0.089608
Epoch: 2 , Training Accuracy: 997/1096 (91%) Training Loss: 0.021502
Epoch: 3 , Training Accuracy: 1036/1096 (95%) Training Loss: 0.013227
Epoch: 4 , Training Accuracy: 1071/1096 (98%) Training Loss: 0.008386
Validating Accuracy: 262/275 (95%)
breakout


In [22]:
trainloader, valloader = load_dataloaders(batch_size=16,start=54, end=60)#73 not included 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1) 
model, optimizer = train(model,device,trainloader,valloader,scheduler,optimizer,criterion,30,1e-3)

file p05_n094
1301
file p04_n103
1301
file p01_n093
1301
file p04_n003
1301
file p05_n078
1301
file p01_n052
1301
Train start
Epoch: 1 , Training Accuracy: 460/728 (63%) Training Loss: 0.091438
Epoch: 2 , Training Accuracy: 658/728 (90%) Training Loss: 0.016368
Epoch: 3 , Training Accuracy: 701/728 (96%) Training Loss: 0.008233
Validating Accuracy: 171/182 (94%)
breakout


In [23]:
trainloader = load_testloader(batch_size=16,start=108, end=112)#73 not included 
test(model,device,trainloader)

file p06_n054
file p05_n047
file p06_n021
file p01_n081
Validate Accuracy: 240/1157 (21%)


In [24]:
trainloader= load_vid(batch_size=16,start=1000, end=3000)#73 not included 
evluate(model,device,trainloader)

tensor([12, 25, 25, 25, 12, 25, 25, 25, 12, 25, 25, 25, 12, 12, 25, 12],
       device='cuda:0')
tensor([25, 12, 12, 25, 10, 25, 25, 25, 12, 12, 25, 12, 25, 25,  1, 25],
       device='cuda:0')
tensor([25, 25, 25, 12, 25, 25, 25, 12, 25, 25, 12, 25, 25, 25, 25, 25],
       device='cuda:0')
tensor([12, 25], device='cuda:0')
